In [144]:
import pandas as pd

df_train = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\train.csv")

df_test = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\test.csv")

df_train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [163]:
import numpy as np

def get_dummies(X, categoricalNumericFeatures=[]):
    # Note the parameter is X, which does not include the label
    for col in X:
        if not pd.api.types.is_numeric_dtype(X[col]) or categoricalNumericFeatures.count(col) > 0:
            X = pd.get_dummies(X, columns=[col], prefix=col, drop_first=True,dummy_na=True)
    return X

def bin_groups(df, categoricalNumericFeatures=[], percent=.05):
    import pandas as pd

    for col in df:
        # if the column is categorical
        if not pd.api.types.is_numeric_dtype(df[col]) or categoricalNumericFeatures.count(col) > 0:
            for group, count in df[col].value_counts().iteritems():
                # For each categorical value, if the percentage of the column that contains that value is below the threshold, relabel to other
                if count / len(df) < percent:
                    df.loc[df[col] == group, col] = 'Other'
    return df

def drop_cols_too_unique(df, percent = .80):

    for col in df:
        pct = len(df[df[col] == 'Other'][col]) / len(df)
        if pct >=.80:
            df.drop(columns=col, inplace=True)
            print("dropped " + col + ", " + str(pct * 100) + "% unique")
    return df        




class MissingData:

    def __init__(self, df, categoricalImputer='', numericalImputer='', categoricalNumericFeatures=[], dropColPct = .5):
        self.categoricalImputer = categoricalImputer
        self.numericalImputer = numericalImputer
        self.df = df
        self.categoricalNumericFeatures = categoricalNumericFeatures
        self.dropColPct = dropColPct

    def fillMissingData(self):
        # get number of categorical and numeric columns missing data
        nCat, nNum = self.missing_values_check(self.df,self.categoricalNumericFeatures)

        # if no columns are missing data, we are done
        if nCat + nNum == 0:
            print("Missing data check complete. No columns have missing data.")
            return
        
        # drop columns missing too much data
        self.df = self.drop_cols_missing_data(self.df,self.dropColPct)

        # handle categorical columns missing data
        if nCat != 0:
            if self.categoricalImputer == 'most_frequent':
                imp = self.impute_mode()
            else:
                raise ValueError("Invalid categorical imputer.")
        
            self.df = self.selective_sklearn_impute(self.df, imp,'categorical',self.categoricalNumericFeatures)
            print(f"imputed {nCat} categorical column(s) using {self.categoricalImputer}.")
        
        # handle numeric columns missing data
        if nNum != 0: 
            if self.numericalImputer == 'mean':
                imp = self.impute_mean()
            elif self.numericalImputer == 'knn':
                imp = self.impute_KNN()
            elif self.numericalImputer == 'reg':
                imp = self.impute_reg()
            else: 
                raise ValueError("Invalid numeric imputer.")
            
            self.df = self.selective_sklearn_impute(self.df,imp,'numeric',self.categoricalNumericFeatures)
            print(f"imputed {nNum} numeric column(s) using {self.numericalImputer}.")

        # Check that filling in the missing data worked
        # get number of categorical and numeric columns missing data
        nCat, nNum = self.missing_values_check(self.df,self.categoricalNumericFeatures)

        # if no columns are missing data, we are done
        if nCat + nNum == 0:
            print("Missing data check complete. No columns have missing data.")
        else:
            raise ValueError("Missing data check failed. Some columns have missing data.")
        return self.df

    def missing_values_report(self):
        df = self.df

        for col in df:
            print(f'{col}\t{round(df[col].isnull().sum() / len(df) * 100,2)}%')

    def missing_values_check(self, df, categoricalNumericFeatures):
        catColsMissingData = 0
        numColsMissingData = 0

        for col in df:
            #if there is at least one null value in the column
            if df[col].isnull().sum() > 0:

                #if its a categorical column
                if not pd.api.types.is_numeric_dtype(df[col]) or col in categoricalNumericFeatures:
                    catColsMissingData += 1
                else:
                    numColsMissingData +=1
        
        return catColsMissingData, numColsMissingData

    def drop_cols_missing_data(self, df, percent=.5):
        for col in df:
            if df[col].isna().sum()/len(df) > percent:
                df = df.drop(columns=col)
                print("dropped the " + col + " column. Over " + str(round(percent*100)) + "% of records were Null." )
        return df

    # IMPUTATION 
    def selective_sklearn_impute(self, df, imp, type, categoricalNumericFeatures = []):
        validTypes = ['categorical', 'numeric']
        if type not in validTypes:
            raise ValueError("Invalid type. Expected one of: %s" % validTypes)
        
        if type == 'categorical':
            # Only impute on categorical columns
            dfCategorical = df.select_dtypes(include='object')
            # if there are categorical features that are numerical, include them (ex: 1, 0 for survived)
            if len(categoricalNumericFeatures) != 0:
                for colName in categoricalNumericFeatures:
                    dfCategorical = dfCategorical.join(df[colName])
            dfCategorical = pd.DataFrame(imp.fit_transform(dfCategorical), columns=dfCategorical.columns)

            # Replace the old categorical columns with the new imputed ones.
            df.drop(columns=dfCategorical.columns, inplace=True)
            df = df.join(dfCategorical)

        elif type == 'numeric':
            # Only impute on numeric columns
            dfNumeric = df.select_dtypes(include='number')
            # if there are categorical features that are numerical, don't include them (ex: 1, 0 for survived)
            if len(categoricalNumericFeatures) != 0:
                for colName in categoricalNumericFeatures:
                    if colName in dfNumeric.columns:
                        dfNumeric = dfNumeric.drop(columns=colName)
            dfNumeric = pd.DataFrame(imp.fit_transform(dfNumeric), columns=dfNumeric.columns)

            # Replace the old numeric columns with the new imputed ones.
            df.drop(columns=dfNumeric.columns, inplace=True)
            df = df.join(dfNumeric)
        return df

    ## CATEGORICAL IMPUTATION
    def impute_mode(self):
        from sklearn.impute import SimpleImputer

        imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

        return imp

    ## NUMERICAL IMPUTATION
    def impute_mean(self):
        from sklearn.impute import SimpleImputer

        # Change the strategy to mean, median, or mode
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')

        return imp

    def impute_KNN(self):
        from sklearn.impute import KNNImputer
        from sklearn.preprocessing import MinMaxScaler

        # Clustering is biased by unstandardized data; so MinMax scale it
        #df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns = df.columns)

        print("did you standardize the data before impute KNN?")
        imp = KNNImputer(n_neighbors=5, weights="uniform")
        return imp

    def impute_reg(self):
        from sklearn.experimental import enable_iterative_imputer
        from sklearn.impute import IterativeImputer

        # Scaling is unnecessary for regression-based imputation
        imp = IterativeImputer(max_iter=10, random_state=12345)
        return imp

  

In [164]:
# Drop columns missing a lot of data (80%).
df = df_train.copy()

missingData = MissingData(df, 'most_frequent','mean', ['Pclass'], .8)
missingData.missing_values_report()

PassengerId	0.0%
Survived	0.0%
Pclass	0.0%
Name	0.0%
Sex	0.0%
Age	19.87%
SibSp	0.0%
Parch	0.0%
Ticket	0.0%
Fare	0.0%
Cabin	77.1%
Embarked	0.22%


### Cabin and Embarked are categorical columns, and right now I will only be using mode imputation for categorical columns. Since imputing mode on cabin will likely not yield any great results, I will drop cabin and continue with imputation

In [165]:
df.drop(columns='Cabin', inplace=True)
df = missingData.fillMissingData()
df.head()

imputed 1 categorical column(s) using most_frequent.
imputed 1 numeric column(s) using mean.
Missing data check complete. No columns have missing data.


,Name,Sex,Ticket,Embarked,Pclass,PassengerId,Survived,Age,SibSp,Parch,Fare
0,"Braund, Mr. Owen Harris",male,A/5 21171,S,3,1.0,0.0,22.0,1.0,0.0,7.2500
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C,1,2.0,1.0,38.0,1.0,0.0,71.2833
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,S,3,3.0,1.0,26.0,0.0,0.0,7.9250
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,S,1,4.0,1.0,35.0,1.0,0.0,53.1000
4,"Allen, Mr. William Henry",male,373450,S,3,5.0,0.0,35.0,0.0,0.0,8.0500


In [248]:
def fit_mlr(df, test_size=.2, random_state=12345, label=''):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    
    X = df.drop(label,axis=1)
    y = df[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model = LinearRegression().fit(X_train, y_train)
    print(f'R-squared (mlr): \t{model.score(X_test, y_test)}')

    return model

def fit_crossvalidate_mlr(df, k, label, repeat=True):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
    from numpy import mean, std

    X = df.drop(label,axis=1)
    y = df[label]

    # prepare the cross-validation procedure
    if repeat:
        cv = RepeatedKFold(n_splits=k, n_repeats=5, random_state=12345)
    else:
        cv = KFold(n_splits=k, random_state=12345, shuffle=True)
    

    # evaluate model
    scores = cross_val_score(LinearRegression(), X, y, scoring='r2', cv=cv, n_jobs=-1)

    # report performance
    print(f'R-squared scores: \n{scores}\n')
    print(f'Average R-squared:\t{mean(scores)}')

    return LinearRegression().fit(X, y)

df2 = df.set_index('PassengerId')

fit_mlr(df2, label='Survived')
model = fit_crossvalidate_mlr(df2, 5, 'Survived', True)

R-squared (mlr): 	0.3647396296033043
R-squared scores: 
[0.36473963 0.3114511  0.44027592 0.43424407 0.36937612 0.32367679
 0.40639937 0.33919578 0.38460849 0.46091504 0.38435369 0.44095512
 0.44551067 0.32924852 0.31186896 0.44691472 0.43244886 0.31745425
 0.40926906 0.28456117 0.40967504 0.33793145 0.30977401 0.44706372
 0.36486114]

Average R-squared:	0.3802709079694058
